---
title: "Comparar medias de dos grupos independientes con t-test en Python"
author: "Traducido y adaptado de R por un asistente de IA"
format: live-html
pyodide: 
    packages: 
        - numpy
        - pandas
        - matplotlib
        - seaborn
        - pingouin
---

## Introducción a las Muestras Independientes vs Pareadas

**Muestra independiente**: Las muestras son independientes si las observaciones en un grupo no están relacionadas con las observaciones en el otro grupo. Por ejemplo, si se mide el peso de dos grupos diferentes de personas (placebo vs tratamiento), los datos de un grupo no están relacionados con los datos del otro grupo.

**Muestra pareada o emparejada**: Las muestras son pareadas si las observaciones en un grupo están relacionadas con las observaciones en el otro grupo. Por ejemplo, si se mide el peso de las mismas personas antes y después de un tratamiento, los datos están relacionados.

En esta lección nos enfocaremos en muestras independientes.

## Ejemplo Práctico

Vamos a trabajar con datos simulados. Tenemos las medidas de peso de dos grupos: mujeres (grupo A) y hombres (grupo B). Queremos saber si la media de peso de las mujeres es significativamente diferente de la de los hombres.

### 1. Cargar librerías y crear datos


```{pyodide}
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pingouin as pg

# Datos simulados
A_mujeres = [38.9, 61.2, 73.3, 21.8, 63.4, 64.6, 48.4, 48.8, 48.5]
B_hombres = [67.8, 60, 63.4, 76, 89.4, 73.3, 67.3, 61.3, 62.4]

# Crear DataFrame
my_data = pd.DataFrame({
    'grupo': ['Mujer'] * len(A_mujeres) + ['Hombre'] * len(B_hombres),
    'pesos': A_mujeres + B_hombres
})

print(my_data)
```

### Condiciones para usar la prueba t de dos muestras independientes

1. **Las dos muestras deben ser independientes**: Esto lo verificamos al diseñar el estudio.
2. **Las muestras deben seguir una distribución normal**: Se puede verificar con la prueba de Shapiro-Wilk.
3. **Las varianzas deben ser iguales (homocedasticidad)**: Se puede verificar con la prueba de Levene.

## Hipótesis de Investigación

- **H₀**: La media de peso de las mujeres es igual a la media de peso de los hombres.
- **H₁**: La media de peso de las mujeres es diferente de la media de peso de los hombres.

## Visualización Preliminar de los Datos

### Gráfico de cajas

```{pyodide}
plt.figure(figsize=(8, 6))
sns.boxplot(x='grupo', y='pesos', data=my_data, palette=['#00AFBB', '#E7B800'])
plt.title('Distribución de pesos por grupo')
plt.ylabel('Peso')
plt.xlabel('Grupos')
plt.show()
```

### Estadísticas descriptivas

```{pyodide}
estadisticas = my_data.groupby('grupo')['pesos'].agg([
    ('cuenta_n', 'count'),
    ('media', 'mean'),
    ('desv_std', 'std')
]).round(3)

print(estadisticas)
```

## Verificación de Supuestos

### 1. Prueba de Normalidad (Shapiro-Wilk)

**Hipótesis**:
- **H₀**: Los datos siguen una distribución normal
- **H₁**: Los datos no siguen una distribución normal

```{pyodide}
# Separar los datos por grupo
mujeres_pesos = my_data[my_data['grupo'] == 'Mujer']['pesos']
hombres_pesos = my_data[my_data['grupo'] == 'Hombre']['pesos']

# Prueba de normalidad para mujeres
stat_mujeres, p_value_mujeres = stats.shapiro(mujeres_pesos)
print(f"Prueba de normalidad - Mujeres:")
print(f"Estadístico: {stat_mujeres:.4f}")
print(f"p-valor: {p_value_mujeres:.4f}")

# Prueba de normalidad para hombres
stat_hombres, p_value_hombres = stats.shapiro(hombres_pesos)
print(f"\nPrueba de normalidad - Hombres:")
print(f"Estadístico: {stat_hombres:.4f}")
print(f"p-valor: {p_value_hombres:.4f}")
```

**Interpretación**: En ambos casos, el p-valor es mayor que 0.05, por lo que no rechazamos la hipótesis nula. Los datos no son significativamente diferentes de una distribución normal.

### 2. Prueba de Homogeneidad de Varianzas (Levene)

**Hipótesis**:
- **H₀**: Las varianzas de los dos grupos son iguales
- **H₁**: Las varianzas de los dos grupos son diferentes

```{pyodide}
# Prueba de Levene para igualdad de varianzas
stat_levene, p_value_levene = stats.levene(mujeres_pesos, hombres_pesos)
print(f"Prueba de Levene:")
print(f"Estadístico: {stat_levene:.4f}")
print(f"p-valor: {p_value_levene:.4f}")

if p_value_levene > 0.05:
    print("No hay evidencia significativa contra la igualdad de varianzas.")
    print("Podemos usar la prueba t clásica (asumiendo varianzas iguales).")
else:
    print("Las varianzas son significativamente diferentes.")
    print("Se recomienda usar la prueba t de Welch (sin asumir varianzas iguales).")
```

## Prueba t de Student

```{pyodide}
# Realizar la prueba t independiente
# equal_var=True asume varianzas iguales (como determinamos arriba)
t_stat, p_value = stats.ttest_ind(mujeres_pesos, hombres_pesos, equal_var=True)

print(f"Prueba t de Student:")
print(f"Estadístico t: {t_stat:.4f}")
print(f"p-valor: {p_value:.4f}")

# Calcular medias de cada grupo
media_mujeres = np.mean(mujeres_pesos)
media_hombres = np.mean(hombres_pesos)
diferencia_medias = media_hombres - media_mujeres

print(f"\nMedia mujeres: {media_mujeres:.3f}")
print(f"Media hombres: {media_hombres:.3f}")
print(f"Diferencia (Hombres - Mujeres): {diferencia_medias:.3f}")

# Intervalo de confianza usando pingouin
resultado_completo = pg.ttest(mujeres_pesos, hombres_pesos, equal_var=True)
print(f"\nIntervalo de confianza (95%): [{resultado_completo['CI95%'].iloc[0][0]:.3f}, {resultado_completo['CI95%'].iloc[0][1]:.3f}]")
```

### Interpretación de Resultados

```{pyodide}
alpha = 0.05
if p_value < alpha:
    print(f"\n✓ CONCLUSIÓN:")
    print(f"El p-valor ({p_value:.4f}) es menor que α = {alpha}")
    print(f"Rechazamos H₀: Hay evidencia significativa de que las medias son diferentes.")
    print(f"Los hombres tienen un peso promedio significativamente diferente al de las mujeres.")
else:
    print(f"\n✗ CONCLUSIÓN:")
    print(f"El p-valor ({p_value:.4f}) es mayor que α = {alpha}")
    print(f"No rechazamos H₀: No hay evidencia suficiente de diferencia en las medias.")
```

## Visualización de Resultados

### Gráfico con estadísticas

```{pyodide}
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Boxplot con puntos individuales
sns.boxplot(x='grupo', y='pesos', data=my_data, palette=['#00AFBB', '#E7B800'], ax=ax1)
sns.stripplot(x='grupo', y='pesos', data=my_data, color='black', alpha=0.6, ax=ax1)
ax1.set_title('Distribución de pesos por grupo')
ax1.set_ylabel('Peso')

# Agregar información del test al gráfico
ax1.text(0.5, 0.95, f'p-valor = {p_value:.4f}', 
         transform=ax1.transAxes, ha='center', va='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Gráfico de barras con error estándar
estadisticas_plot = my_data.groupby('grupo')['pesos'].agg(['mean', 'sem']).reset_index()
bars = ax2.bar(estadisticas_plot['grupo'], estadisticas_plot['mean'], 
               yerr=estadisticas_plot['sem'], capsize=5, 
               color=['#00AFBB', '#E7B800'], alpha=0.7)
ax2.set_title('Medias con Error Estándar')
ax2.set_ylabel('Peso Promedio')

# Agregar valores en las barras
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + estadisticas_plot['sem'].iloc[i],
             f'{height:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()
```

## Prueba t de Welch (alternativa cuando las varianzas son diferentes)

Si las varianzas no fueran iguales, usaríamos la prueba t de Welch:

```{pyodide}
# Prueba t de Welch (sin asumir varianzas iguales)
t_stat_welch, p_value_welch = stats.ttest_ind(mujeres_pesos, hombres_pesos, equal_var=False)

print(f"Prueba t de Welch (varianzas diferentes):")
print(f"Estadístico t: {t_stat_welch:.4f}")
print(f"p-valor: {p_value_welch:.4f}")
```

## Tamaño del Efecto (Cohen's d)

El tamaño del efecto nos dice qué tan grande es la diferencia práctica entre los grupos:

```{pyodide}
# Calcular Cohen's d usando pingouin
cohens_d = pg.compute_effsize(mujeres_pesos, hombres_pesos, eftype='cohen')
print(f"Tamaño del efecto (Cohen's d): {cohens_d:.3f}")

# Interpretación de Cohen's d
if abs(cohens_d) < 0.2:
    interpretacion = "efecto pequeño"
elif abs(cohens_d) < 0.5:
    interpretacion = "efecto pequeño"
elif abs(cohens_d) < 0.8:
    interpretacion = "efecto mediano"
else:
    interpretacion = "efecto grande"

print(f"Interpretación: {interpretacion}")
```

## Resumen de la Metodología

1. **Verificar supuestos**: Normalidad (Shapiro-Wilk) y homogeneidad de varianzas (Levene)
2. **Elegir la prueba apropiada**: t-test clásica (varianzas iguales) vs t-test de Welch (varianzas diferentes)
3. **Interpretar resultados**: p-valor, diferencia de medias, intervalo de confianza
4. **Evaluar significancia práctica**: Tamaño del efecto (Cohen's d)
5. **Visualizar resultados**: Gráficos que muestren tanto la distribución como las estadísticas